In [1]:
import pandas as pd
import time
import datetime
import plotly.express as px
from pathlib import Path
import torch
from lib.models import MLP2hl
from lib.modules import evaluate_loop, plot_and_save_cm, summary, window_session, pad_for_windowing, predict_and_plot_pretty_session
from lib.utils import get_bouts_smoothed
from torch import nn
from torch.utils.data import DataLoader, TensorDataset

In [2]:
WINSIZE = 101
DEVICE = 'cuda:1'

In [3]:
data_dir = Path('../data/andrew/2023-10-26_15_32_20/')
acceleration = pd.read_csv(data_dir / 'acceleration.csv',skiprows=1).rename({'x':'x_acc', 'y':'y_acc', 'z':'z_acc'}, axis=1)
acceleration_start_time_seconds = float(pd.read_csv(data_dir / 'acceleration.csv',nrows=1,header=None).iloc[0,0].split()[-1])/1000
acceleration.timestamp = ((acceleration.timestamp - acceleration.timestamp[0])*1e-9)+acceleration_start_time_seconds
acceleration = acceleration.dropna()

start = int(datetime.datetime(2023, 10, 26, 16, 20, 0).strftime('%s'))
end = int(datetime.datetime(2023, 10, 26, 16, 37, 0).strftime('%s'))
acceleration['label'] = 0
acceleration.loc[(acceleration.timestamp > start) & (acceleration.timestamp < end),'label'] = 1

print(session_dir.name)
print(len(acceleration))
print(f'{timedelta(seconds=acceleration.timestamp.iloc[-1] - acceleration.timestamp.iloc[0])}')

In [4]:
Xte = torch.Tensor(acceleration[['x_acc','y_acc','z_acc']].values)
yte = torch.Tensor(acceleration['label'].values).unsqueeze(1)

Xte = pad_for_windowing(Xte, WINSIZE)
Xte = window_session(Xte, WINSIZE)

In [5]:
model = MLP2hl([20,20], WINSIZE).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()

In [12]:
model.load_state_dict(torch.load(Path('dev/mlp2hl/best_model.pt')))
# model.load_state_dict(torch.load(Path('dev/mlp2hl_3/model/1.pt')))

<All keys matched successfully>

In [13]:
testloader = DataLoader(TensorDataset(Xte,yte), batch_size=64)
ys,metrics = evaluate_loop(model, criterion, testloader, DEVICE)
plot_and_save_cm(ys['true'], ys['pred'])
summary(metrics)

KeyboardInterrupt: 

In [ ]:
# Cut off part of session
buffer = 700_000
dim_factor = 10

first_pos = max(acceleration.loc[acceleration['label']==1].index[0]-buffer, 0)
last_pos = min(acceleration.loc[acceleration['label']==1].index[-1]+buffer, len(acceleration))

session = acceleration.loc[first_pos:last_pos-1, ['x_acc','y_acc','z_acc', 'label']].reset_index(drop=True)
session['Predicted Eating'] = ys['pred'][first_pos:last_pos]
session['Confidence'] = ys['conf'][first_pos:last_pos]

pred_bouts = get_bouts_smoothed(ys['pred'][first_pos:last_pos])

fig = px.line(
    session[::dim_factor], 
    x=session.index[::dim_factor], 
    y=['x_acc','y_acc','z_acc', 'Predicted Eating', 'Confidence']
)
fig.add_vrect(
    x0=session.loc[session['label']==1].index[0], 
    x1=session.loc[session['label']==1].index[-1], 
    fillcolor='black', 
    opacity=.2,
    line_width=0,
    layer="below"
)
for bout in pred_bouts:
    fig.add_vrect(
        x0=bout['start'], 
        x1=bout['end'], 
        fillcolor='red', 
        opacity=.2,
        line_width=0,
        layer="below"
    )
fig.show(renderer='browser')

In [ ]:
fig.write_html('dev/mlp2hl_andrew_plot.html')